In [1]:
import numpy as np
import pandas as pd
import json
import os
import time
from tqdm.auto import tqdm

In [2]:
# event_file_path = ".\\events"
# movement_file_path = "."

event_file_path = "C:\\WuYihong\\Data\\nba_movement_data\\data\\events"
movement_file_path = "C:\\WuYihong\\Data\\nba_movement_data\\data"

event_file_list = list(filter(lambda x: x.endswith(".csv"), os.listdir(event_file_path)))
movement_file_list = list(filter(lambda x: x.endswith(".json"), os.listdir(movement_file_path)))

# event_file_list, movement_file_list

In [72]:
with open("game_info\\court_info_new.json", "r") as f:
    data = json.load(f)

total_wrong = 0
wrong_list = []
for key, value in data.items():
    if value['home'] == -1 or value['visitor'] == -1:
        print("MISS", key)
        total_wrong += 1
        continue
    
    if value['home'][0] == value['visitor'][0]:
        print("OVERLAP", key)
        wrong_list.append(key)
        total_wrong += 1

total_wrong, wrong_list

OVERLAP 0021500049
OVERLAP 0021500091
OVERLAP 0021500134
OVERLAP 0021500195
OVERLAP 0021500265
OVERLAP 0021500266
OVERLAP 0021500295
OVERLAP 0021500296
OVERLAP 0021500353
OVERLAP 0021500386
OVERLAP 0021500470
OVERLAP 0021500473
OVERLAP 0021500499
OVERLAP 0021500511
OVERLAP 0021500525
OVERLAP 0021500532
OVERLAP 0021500545
OVERLAP 0021500551
OVERLAP 0021500571
OVERLAP 0021500573
OVERLAP 0021500579
OVERLAP 0021500581
OVERLAP 0021500594
OVERLAP 0021500625
OVERLAP 0021500626
OVERLAP 0021500637
OVERLAP 0021500651
OVERLAP 0021500657
OVERLAP 0021500659
OVERLAP 0021500660


(30,
 ['0021500049',
  '0021500091',
  '0021500134',
  '0021500195',
  '0021500265',
  '0021500266',
  '0021500295',
  '0021500296',
  '0021500353',
  '0021500386',
  '0021500470',
  '0021500473',
  '0021500499',
  '0021500511',
  '0021500525',
  '0021500532',
  '0021500545',
  '0021500551',
  '0021500571',
  '0021500573',
  '0021500579',
  '0021500581',
  '0021500594',
  '0021500625',
  '0021500626',
  '0021500637',
  '0021500651',
  '0021500657',
  '0021500659',
  '0021500660'])

In [73]:
game_list = ['0021500049',
    '0021500091',
    '0021500134',
    '0021500195',
    '0021500265',
    '0021500266',
    '0021500295',
    '0021500296',
    '0021500353',
    '0021500386',
    '0021500470',
    '0021500473',
    '0021500499',
    '0021500511',
    '0021500525',
    '0021500532',
    '0021500545',
    '0021500551',
    '0021500571',
    '0021500573',
    '0021500579',
    '0021500581',
    '0021500594',
    '0021500625',
    '0021500626',
    '0021500637',
    '0021500651',
    '0021500657',
    '0021500659',
    '0021500660']

In [77]:
# extract events and times
court_info = {}
# for event, movement in zip(event_file_list, movement_file_list):
for game_id in game_list:
    # event = "0021500134.csv"
    # movement = "0021500134.json"
    event = "{}.csv".format(game_id)
    movement = "{}.json".format(game_id)

    event_file = os.path.join(event_file_path, event)
    movement_file = os.path.join(movement_file_path, movement)

    event_df = pd.read_csv(event_file)

    start_time = time.time()
    print("")
    court_dict = confirm_half_court(event_file, movement_file)
    game_id = event.split(".")[0]
    print(game_id, court_dict, "Time spend:", time.time() - start_time)
    print("")

    court_info[game_id] = court_dict

    # break

    with open("game_info\\court_info_new.json", "w") as f:
        json.dump(court_info, f, indent=4)


[[30, 498, 0], [35, 456, 0], [39, 431, 0]] [[33, 467, 1], [45, 404, 1], [52, 376, -1]]
0021500049 {'home': [0, 47], 'visitor': [0, 47]} Time spend: 4.4630818367004395


[[26, 487, 0], [37, 456, 1], [38, 416, 0]] [[30, 463, 1], [40, 403, -1], [45, 375, 1]]
0021500091 {'home': [0, 47], 'visitor': [0, 47]} Time spend: 4.517634153366089


[[28, 495, 1], [38, 418, -1], [43, 371, 1]] [[32, 477, 0], [42, 395, 0], [47, 331, 0]]
0021500134 {'home': [0, 47], 'visitor': [0, 47]} Time spend: 5.814282417297363


[[30, 492, 0], [40, 427, 0], [43, 404, 0]] [[38, 445, -1], [45, 388, 1], [48, 351, 1]]
0021500195 {'home': [0, 47], 'visitor': [0, 47]} Time spend: 5.0575950145721436


[[32, 476, -1], [36, 453, 1], [51, 396, 1]] [[29, 491, 0], [34, 464, 0], [38, 446, 0]]
0021500265 {'home': [0, 47], 'visitor': [0, 47]} Time spend: 4.481085300445557


[[28, 460, 0], [41, 389, 0], [59, 308, 0]] [[39, 406, -1], [52, 351, 1], [57, 316, 1]]
0021500266 {'home': [0, 47], 'visitor': [0, 47]} Time spend: 4.3779773

In [74]:
def convert_time(time_string):
    minute = int(time_string.split(":")[0])
    second = int(time_string.split(":")[1])
    
    return minute * 60 + second

# confirm offensive court of both teams
def confirm_half_court(event_file, movement_file):
    event_df = pd.read_csv(event_file)
    court_dict = {"home": -1, "visitor": -1}
    home_info = []
    visitor_info = []
    movement_data = {}

    with open(movement_file, "r") as f:
        movement_data = json.load(f)

    for event_id in range(len(event_df)):
        event_item = event_df.loc[event_id]

        if isinstance(event_item.HOMEDESCRIPTION, float) and isinstance(event_item.VISITORDESCRIPTION, float):
            continue
        
        period = event_item.PERIOD
        if int(period) > 1:
            print("Errors in confirming half court period.", event_file)
            break
        
        if not isinstance(event_item.HOMEDESCRIPTION, float):
            if "shot" in event_item.HOMEDESCRIPTION.lower():
                event_id = event_item.EVENTNUM
                timestamp = convert_time(event_item.PCTIMESTRING)
                if timestamp < 500 and len(home_info) < 3:
                    home_info.append([event_id, timestamp, -1])
                
        if not isinstance(event_item.VISITORDESCRIPTION, float):
            if "shot" in event_item.VISITORDESCRIPTION.lower():
                event_id = event_item.EVENTNUM
                timestamp = convert_time(event_item.PCTIMESTRING)
                if timestamp < 500 and len(visitor_info) < 3:
                    visitor_info.append([event_id, timestamp, -1])
        
        if len(visitor_info) == 3 and len(home_info) == 3:
            # print(home_info, visitor_info)
            break
    
    for event in movement_data['events']:
        check_flag = True
        if len(event['moments']) < 5:
            continue

        if int(event['moments'][0][0]) != 1:
            # print("Errors in second confirming half court period.", event_file)
            continue
        
        for item in home_info:
            if item[-1] != -1:
                continue
            if event['moments'][0][2] + 1 > item[1] > event['moments'][-1][2] - 2:
                for frame in event['moments']:
                    game_clock = frame[2]

                    if abs(game_clock - item[1]) < 1:
                        ball_x = frame[5][0][2]
                        if ball_x > 47:
                            item[-1] = 1
                        else:
                            item[-1] = 0
                        
                        break
                    
        for item in visitor_info:
            if item[-1] != -1:
                continue
            
            if event['moments'][0][2] + 1 > item[1] > event['moments'][-1][2] - 2:
                for frame in event['moments']:
                    game_clock = frame[2]

                    if abs(game_clock - item[1]) < 1:
                        match_flag = False
                        ball_x = frame[5][0][2]
                        if ball_x > 47:
                            item[-1] = 1
                        else:
                            item[-1] = 0
                        
                        break
        
        for item in home_info:
            if item[-1] == -1:
                check_flag = False
        for item in visitor_info:
            if item[-1] == -1:
                check_flag = False
        
        if check_flag:
            break
    
    home_result = 0
    visitor_result = 0
    for item in home_info:
        home_result += item[-1]
    if home_result >= 2:
        court_dict['home'] = [47, 94]
    elif home_result <= 1:
        court_dict['home'] = [0, 47]
    else:
        print("Errors in home!")

    for item in visitor_info:
        visitor_result += item[-1]
    if visitor_result >= 2:
        court_dict['visitor'] = [47, 94]
    elif visitor_result <= 1:
        court_dict['visitor'] = [0, 47]
    else:
        print("Errors in visitor!")

    if home_result not in (0, 3) or visitor_result not in (0, 3):
        print(home_info, visitor_info)

    return court_dict 